<a href="https://colab.research.google.com/github/Lagos1989s/titanic-analysis/blob/main/Task_03_%E9%90%B5%E9%81%94%E5%B0%BC%E8%99%9F%E5%AD%98%E6%B4%BB%E9%A0%90%E6%B8%AC%E5%B0%88%E9%A1%8C%E5%AF%A6%E4%BD%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd
import numpy as np

df = pd.read_csv('https://raw.githubusercontent.com/dsindy/kaggle-titanic/master/data/train.csv')

In [49]:
# 1. 在資料分析的工作流程中，在不考慮準確度情況下有哪些必備的環節?

# i.   確認每個資料欄位的定義，觀察欄位與欄位之間的關係。
# ii.  確認每個欄位的資料格式、以及是否有遺漏值。
# iii. 根據已知的資訊以及依據欄位之間的關係，盡可能地將遺漏值補回。
# iv.  依據類別欄位的特性，分別進行Label Encoding或是One-hot Encoding處理。
# v.   確認是否還有其他欄位是模型無法讀取的，再另外找出處理的方法。

In [48]:
# Titanic 資料欄位定義

# PassengerId: 乘客編號
# survival:    是否生存，0:死亡 1:生存。
# pclass:      社會經濟地位替代指標，1:高 2:中 3:低。
# sex:         性別。
# Age:         年齡。
# sibsp:       在鐵達尼號上兄弟姊妹或配偶的數量。
# parch:       在鐵達尼號上父母或小孩的數量。
# ticket:      船票編號。
# fare:        乘客票價。
# cabin:       艙位編號。
# embarked:    登船港口, C = Cherbourg, Q = Queenstown, S = Southampton。

In [9]:
# 2. 不考慮準確度的前提下完成 Baseline ，讓已讀入的資料完成必要的資料前處理的操作。
# 觀察資料狀況，如各欄位的統計指標、欄位的資料格式以及是否有遺漏值。

print (df.describe())
print ('\n----------------------------------------\n')
print (df.info())
print ('\n----------------------------------------\n')
print (df.isnull().sum())

       PassengerId    Survived      Pclass         Age       SibSp  \
count   891.000000  891.000000  891.000000  714.000000  891.000000   
mean    446.000000    0.383838    2.308642   29.699118    0.523008   
std     257.353842    0.486592    0.836071   14.526497    1.102743   
min       1.000000    0.000000    1.000000    0.420000    0.000000   
25%     223.500000    0.000000    2.000000   20.125000    0.000000   
50%     446.000000    0.000000    3.000000   28.000000    0.000000   
75%     668.500000    1.000000    3.000000   38.000000    1.000000   
max     891.000000    1.000000    3.000000   80.000000    8.000000   

            Parch        Fare  
count  891.000000  891.000000  
mean     0.381594   32.204208  
std      0.806057   49.693429  
min      0.000000    0.000000  
25%      0.000000    7.910400  
50%      0.000000   14.454200  
75%      0.000000   31.000000  
max      6.000000  512.329200  

----------------------------------------

<class 'pandas.core.frame.DataFrame'>


In [15]:
# Age、Cabin、Embarked的遺漏值處理

# 以不同性別的平均年齡補回年齡遺漏值。
# Age
df['Age'].fillna(value=df.groupby('Sex')['Age'].transform('mean'), inplace=True)

# Embarked
# 將Embarked轉為數值後，以眾數S補回。
print (df['Embarked'].mode())
df['Embarked'].replace(np.nan, 'S', inplace=True)

# Cabin
# 將Cabin為Nan的值以No_Cabin標記
df['Cabin'].replace(np.nan, 'No_Cabin', inplace=True)

0    S
Name: Embarked, dtype: object


In [41]:
# 將Ticket欄位分隔為Ticket_item以及Ticket_number
def ticket_number(x):
  return x.split(" ")[-1]

def ticket_item(x):
  items = x.split(" ")
  if len(items)==1:
    return 'None'
  return '_'.join(items[0:-1])

df['Ticket_number'] = df['Ticket'].apply(ticket_number)
df['Ticket_item'] = df['Ticket'].apply(ticket_item)

# 處理Ticket_number有非數值之資料
# 將LINE處理為0
df['Ticket_number'].replace('LINE', 0, inplace=True)
df.Ticket_number = df.Ticket_number.astype('int64')

In [43]:
from sklearn import preprocessing

# 類別型的欄位編碼處理
# 對Sex欄位進行Label Encoding
le = preprocessing.LabelEncoder()
df['Sex'] = le.fit_transform(df['Sex'])

In [44]:
# 將欄位區分成已經處理好的欄位以及還需要處理的類別變數之DataFrame，處理完畢後合併。
# 已處理完畢的欄位
column_fin = ['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Ticket_number']
column_dum = ['Cabin', 'Embarked', 'Ticket_item']

# 針對類別型的欄位進行One-hot Encoding
df_fin = df[column_fin]
df_dum = pd.get_dummies(df[column_dum])

# 將兩個處理好的DataFrame合併
df_train = pd.concat([df_fin, df_dum], axis=1)

In [46]:
columns_X = set(df_train.columns) - {'Survived'}
columns_y = ['Survived']

train_X = df_train[columns_X]
train_y = df_train[columns_y]

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

log = LogisticRegression(random_state=0, max_iter=3000)
scores = cross_val_score(log, train_X, train_y.values.ravel(), cv=5, scoring='accuracy')
print(scores)

[0.60893855 0.61797753 0.6741573  0.61797753 0.61235955]


<ipython-input-46-2e2101e53858>:4: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  train_X = df_train[columns_X]
